In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

# 1. Cargar datos
data_path = "Life_Expectancy_Data.xlsx"  # ajusta la ruta si es necesario
df = pd.read_excel(data_path)

# 2. Limpiar nombres de columnas
df.columns = [c.strip() for c in df.columns]

# 3. Detectar la columna de esperanza de vida (Life expectancy)
target_candidates = [
    c for c in df.columns
    if "life" in c.lower() and "expect" in c.lower()
]

if not target_candidates:
    raise ValueError("No se encontró una columna que parezca 'Life expectancy' en el archivo.")

target_col = target_candidates[0]

# 4. Eliminar filas sin dato de esperanza de vida
df = df.dropna(subset=[target_col])

# 5. Quitar 'Status' si existe (simplificación)
if "Status" in df.columns:
    df = df.drop(columns=["Status"])

# 6. Construir X (features) e y continua
X = df.drop(columns=[target_col])
y_cont = df[target_col]

# 7. One-Hot Encoding de Country (si está presente)
if "Country" in X.columns:
    X = pd.get_dummies(X, columns=["Country"], drop_first=True)

# 8. Rellenar NaN numéricos con la mediana
numeric_cols = X.select_dtypes(include=[np.number]).columns
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())

# 9. Crear variable objetivo categórica (mismas clases que en KNN)
class_labels = [
    "Esperanza de vida muy baja",
    "Esperanza de vida baja",
    "Esperanza de vida media",
    "Esperanza de vida alta",
    "Esperanza de vida muy alta"
]
y = pd.qcut(y_cont, q=5, labels=class_labels)

# 10. División train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 11. Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 12. Definir y entrenar el MLP (estructura sencilla)
mlp = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation="relu",
    solver="adam",
    max_iter=600,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)

# 13. Evaluación
y_pred = mlp.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred, labels=class_labels)

print("RESULTADOS MLP - CLASIFICACIÓN ESPERANZA DE VIDA")
print("------------------------------------------------")
print(f"Exactitud (accuracy): {acc:.4f}")
print("\nMatriz de confusión (filas = real, columnas = predicho):")
print(pd.DataFrame(cm, index=class_labels, columns=class_labels))

# 14. Guardar artefactos para el backend
joblib.dump(mlp, "life_mlp_model.pkl")
joblib.dump(scaler, "life_mlp_scaler.pkl")
joblib.dump(class_labels, "life_mlp_classes.pkl")
joblib.dump(X.columns.tolist(), "life_mlp_features.pkl")

print("\nArchivos guardados:")
print("- life_mlp_model.pkl")
print("- life_mlp_scaler.pkl")
print("- life_mlp_classes.pkl")
print("- life_mlp_features.pkl")


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


Probando MLP (32,), alpha=0.0001: accuracy = 0.8959


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


Probando MLP (64,), alpha=0.0001: accuracy = 0.8942
Probando MLP (32, 16), alpha=0.0001: accuracy = 0.8925
Probando MLP (64, 32), alpha=0.0001: accuracy = 0.8857
Probando MLP (32, 16), alpha=0.001: accuracy = 0.9010
Probando MLP (64, 32), alpha=0.001: accuracy = 0.8908

Mejor configuración encontrada:
hidden_layer_sizes = (32, 16), alpha = 0.001
Accuracy validación = 0.9010

RESULTADOS FINALES MLP - ESPERANZA DE VIDA
-------------------------------------------
Accuracy final: 0.9010

Matriz de confusión (filas = real, columnas = predicho):
                            Esperanza de vida muy baja  \
Esperanza de vida muy baja                         114   
Esperanza de vida baja                               7   
Esperanza de vida media                              1   
Esperanza de vida alta                               0   
Esperanza de vida muy alta                           0   

                            Esperanza de vida baja  Esperanza de vida media  \
Esperanza de vida muy baja